# Import

In [1]:
import pandas as pd
import numpy as np
from node import node
from multiprocessing import Pool, cpu_count, Process
import os

# Console run

In [45]:
S_A = pd.read_csv("EM/Input/S_A.csv", header=0, index_col=0)
S_D = pd.read_csv("EM/Input/S_D.csv", header=0, index_col=0)
A_D = pd.read_csv("EM/Input/A_Do.csv", index_col=0, header=0)

In [2]:
A_D = pd.read_csv("chunhui/Input/HNSC/DriverDEGmatrix.HNSC.csv",
                  header=0,
                  index_col=0)
S_D = pd.read_csv("chunhui/Input/HNSC/DEGmatrix.HNSC.csv",
                  header=0,
                  index_col=0)
S_A = pd.read_csv("chunhui/Input/HNSC/SGAmatrix.HNSC.csv",
                  header=0,
                  index_col=0)

S_A.index = [ele[0:12] for ele in S_A.index]
S_D.index = [ele[0:12] for ele in S_D.index]

In [3]:
import fges_mcmc_py
import importlib

importlib.reload(fges_mcmc_py)

for i in range(1):
    print(i)
    sbtype = "HNSC"
    file_name = "clu"
    #     file_name = "HNSC_%s" % i
    S_P = pd.read_csv("chunhui/Output/%s/S_Pci_%s.csv" % (sbtype, file_name),
                      header=0,
                      index_col=0)
    S_P.index = [ele[0:12] for ele in S_P.index]
    S_Ai, S_PD, sga_l, A_Di = fges_mcmc_py.data_pp(S_A.loc[S_P.index], S_P,
                                                   S_D.loc[S_P.index], A_D,
                                                   S_P.columns)

    fges_mcmc_py.create_knowledge(S_Ai.columns, A_Di,
                                  "./Pathway/BI/knowledge_%s.txt" % file_name)

    S_PD.to_csv("./Pathway/BI/S_Pci_%s.csv" % file_name,
                index=False,
                header=True)

    os.system(
        "java -jar causal-cmd-1.1.3-jar-with-dependencies.jar --algorithm fges \
        --score disc-bic-score --dataset ./Pathway/BI/S_Pci_%s.csv --data-type discrete --delimiter comma \
        --faithfulnessAssumed True --symmetricFirstStep True \
        --verbose True --out ./Pathway/BI/ --knowledge ./Pathway/BI/knowledge_%s.txt "
        % (file_name, file_name))


ModuleNotFoundError: No module named 'javabridge'

# Cycle Run

In [ ]:
fges_mcmc_py.fges_mcmc(
    S_A.loc[S_P.index],
    S_P,
    S_D.loc[S_P.index],
    A_D,
    S_P.columns,
    "Pathway/BI/S_P_HNSC",
    10,
)


In [ ]:
S_P = pd.read_csv("para_test.csv", index_col=0, header=0).T
S_P.index = [ele[0:-3] for ele in S_P.index]

S_Pn = S_P / S_P.max(axis=0)
S_Pn[S_Pn > 0.3] = 1
S_Pn[abs(S_Pn) <= 0.3] = 0
S_Pn[S_Pn < 0.3] = -1

A_D = pd.read_csv("EM/Input/A_Do.csv", index_col=0, header=0)

# Multiple Run

In [6]:
import fges_mcmc_py
import importlib

importlib.reload(fges_mcmc_py)

A_D = pd.read_csv("EM/Input/A_Do.csv", header=0, index_col=0)

p = Pool(10)
for j in range(10):

    S_Pn = pd.read_csv("EM/Output/BI/S_P%s" % j, header=0, index_col=0)
    S_Pn = S_Pn > 0.5

    p.apply_async(fges_mcmc_py.fges_mcmc,
                  args=(
                      S_A.loc[S_Pn.index],
                      S_Pn,
                      S_D.loc[S_Pn.index],
                      A_D,
                      S_Pn.columns,
                      "Pathway/BI/S_P%s" % j,
                      10,
                  ))

p.close()
p.join()
p.terminate()

KeyboardInterrupt: 